In [1]:
%matplotlib inline
import scanpy as sc
import scvi
import numpy as np
from sctools import integrate, plot

Global seed set to 0


## Data integration using SCVI
This step aims to harmonize the data and remove batch effects

In [2]:
prefixes = ['tissue.scps', 'tissue.ad', 'tissue.uc', 'pbmc.scps']
adatas = {
    k: sc.read_h5ad(f'../data/{k}.qcfiltered.h5ad') for k in prefixes
}

adatas

{'tissue.scps': AnnData object with n_obs × n_vars = 332546 × 20912
     obs: 'sample_id', 'patient_id', 'status', 'tissue', 'cell_fraction', 'doublet', 'doublet_score', 'nFeature_RNA', 'nCount_RNA', 'percent_mt', 'percent_ribo', 'qc_pass',
 'tissue.ad': AnnData object with n_obs × n_vars = 318976 × 21750
     obs: 'sample_id', 'patient_id', 'status', 'tissue', 'cell_fraction', 'doublet', 'doublet_score', 'nFeature_RNA', 'nCount_RNA', 'percent_mt', 'percent_ribo', 'qc_pass',
 'tissue.uc': AnnData object with n_obs × n_vars = 79378 × 18436
     obs: 'sample_id', 'patient_id', 'status', 'tissue', 'cell_fraction', 'doublet', 'doublet_score', 'nFeature_RNA', 'nCount_RNA', 'percent_mt', 'percent_ribo', 'qc_pass',
 'pbmc.scps': AnnData object with n_obs × n_vars = 40148 × 15998
     obs: 'sample_id', 'patient_id', 'status', 'tissue', 'cell_fraction', 'doublet', 'doublet_score', 'nFeature_RNA', 'nCount_RNA', 'percent_mt', 'percent_ribo', 'qc_pass'}

In [13]:
adata = adatas['tissue.scps']
cells_per_patient = adata.obs.groupby(['status', 'patient_id']).count()
cell_count_table = cells_per_modality.drop(
    index = cells_per_modality.index[~(cells_per_modality > 0).all(axis = 1)]
)
cell_count_table[['sample_id']].to_csv('../data/cell_counts_per_patient.csv')

In [3]:
# visualize raw data
for k, adata in adatas.items():
    print(k)
    plot.integrate.raw_data_umap(
        adata,
        ['status', 'tissue', 'FOXP3'],
        size = 10,
        vmax = 0.5,
        savefile = f'../plots/{k}.raw.umap.png'
    )

tissue.scps


/users/daniel.malzl/.local/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:64: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  warnings.warn(
/users/daniel.malzl/.local/lib/python3.9/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/users/daniel.malzl/.local/lib/python3.9/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/st

tissue.ad


/users/daniel.malzl/.local/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:64: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  warnings.warn(
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
/users/daniel.malzl/.local/lib/python3.9/site-packages/scanpy/plotting/_tools/scatterplots.py:394: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(
/users/daniel.malzl/.local/lib/python3.9/site-packages/scanpy/plotting/_tools/scatterplots.py:394: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(


tissue.uc


/users/daniel.malzl/.local/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:64: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  warnings.warn(
/users/daniel.malzl/.local/lib/python3.9/site-packages/scanpy/plotting/_tools/scatterplots.py:394: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(
/users/daniel.malzl/.local/lib/python3.9/site-packages/scanpy/plotting/_tools/scatterplots.py:394: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(


pbmc.scps


/users/daniel.malzl/.local/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:64: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  warnings.warn(
/users/daniel.malzl/.local/lib/python3.9/site-packages/scanpy/plotting/_tools/scatterplots.py:394: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(
/users/daniel.malzl/.local/lib/python3.9/site-packages/scanpy/plotting/_tools/scatterplots.py:394: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(


In [ ]:
integration_params = {
    k: {'kwargs': dict()} for k in adatas.keys()
}

integration_results = {}
for key, adata in adatas.items():
    print(key)
    params = integration_params[key]
    integration_results[key] = integrate.integrate_data_scvi(
        adata[params['subset'], :].copy(),
        'sample_id',
        train_size = 1,
        **params['kwargs']
    )

    integration_results[key]['data'].write(
        f'../data/{key}.integrated.h5ad'
    )

    integration_results[key]['model'].save(
        f'../data/{key}.integration.scvi.model',
        overwrite = True
    )

In [2]:
# restore results if kernel breaks or gets shut down
integration_results = {}
for key in ['tissue.scps', 'tissue.ad', 'tissue.uc', 'pbmc.scps']:
    data = sc.read_h5ad(
        f'../data/{key}.integrated.h5ad'
    )
    integration_results[key] = {
        'data': data,
        'model': scvi.model.SCVI.load(
            f'../data/{key}.integration.scvi.model', 
            adata = data
        )
    }
integration_results

/users/daniel.malzl/.conda/envs/scpython/lib/python3.9/abc.py:98: FutureWarning: SparseDataset is deprecated and will be removed in late 2024. It has been replaced by the public classes CSRDataset and CSCDataset.

For instance checks, use `isinstance(X, (anndata.experimental.CSRDataset, anndata.experimental.CSCDataset))` instead.

For creation, use `anndata.experimental.sparse_dataset(X)` instead.

  return _abc_instancecheck(cls, instance)


SCVI Model with the following params: 
n_hidden: 128, n_latent: 30, n_layers: 2, dropout_rate: 0.1, dispersion: gene, gene_likelihood: nb, 
latent_distribution: normal
Training status: Trained

SCVI Model with the following params: 
n_hidden: 128, n_latent: 30, n_layers: 2, dropout_rate: 0.1, dispersion: gene, gene_likelihood: nb, 
latent_distribution: normal
Training status: Trained

SCVI Model with the following params: 
n_hidden: 128, n_latent: 30, n_layers: 2, dropout_rate: 0.1, dispersion: gene, gene_likelihood: nb, 
latent_distribution: normal
Training status: Trained

SCVI Model with the following params: 
n_hidden: 128, n_latent: 30, n_layers: 2, dropout_rate: 0.1, dispersion: gene, gene_likelihood: nb, 
latent_distribution: normal
Training status: Trained

{'tissue.scps': {'data': AnnData object with n_obs × n_vars = 332546 × 4000
      obs: 'sample_id', 'patient_id', 'status', 'tissue', 'cell_fraction', 'doublet', 'doublet_score', 'nFeature_RNA', 'nCount_RNA', 'percent_mt', 'percent_ribo', 'qc_pass', '_scvi_batch', '_scvi_labels'
      var: 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
      uns: '_scvi_manager_uuid', '_scvi_uuid', 'hvg', 'neighbors', 'umap'
      obsm: 'X_scvi', 'X_umap'
      layers: 'counts'
      obsp: 'connectivities', 'distances',
  'model': },
 'tissue.ad': {'data': AnnData object with n_obs × n_vars = 318976 × 4000
      obs: 'sample_id', 'patient_id', 'status', 'tissue', 'cell_fraction', 'doublet', 'doublet_score', 'nFeature_RNA', 'nCount_RNA', 'percent_mt', 'percent_ribo', 'qc_pass', '_scvi_batch', '_scvi_labels'
      var: 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
      uns: '_scvi_manager_uuid', '_scvi_uuid', 'hvg', 'neighbors', 'uma

In [3]:
fig, axs = plot.integrate.plot_integration_results(
    integration_results,
    ['status', 'sample_id', 'FOXP3', 'CD3D'],
    [
        dict(size = 10, vmax = None),
        dict(size = 10, vmax = None),
        dict(size = 10, vmax = 1),
        dict(size = 10, vmax = 10)
    ],
    data_key = 'data',
    legend_off = True
)

/users/daniel.malzl/.local/lib/python3.9/site-packages/scanpy/plotting/_utils.py:432: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[value_to_plot + '_colors'] = colors_list
/users/daniel.malzl/.local/lib/python3.9/site-packages/scanpy/plotting/_tools/scatterplots.py:394: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(
/users/daniel.malzl/.local/lib/python3.9/site-packages/scanpy/plotting/_utils.py:432: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[value_to_plot + '_colors'] = colors_list
/users/daniel.malzl/.local/lib/python3.9/site-packages/scanpy/plotting/_tools/scatterplots.py:394: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are

## Identification of T-cell subsets
This section aims at extracting T-cells from full dataset based on CD3D and FOXP3 expression (FOXP3 is taken into account here because the ultimate goal is to extract regulatory T-cells)

In [4]:
fig, axs = plot.integrate.plot_leiden_clusterings(
    integration_results,
    [0.1, 0.25, 0.3, 0.4],
    data_key = 'data',
    legend_loc = 'on data'
)
fig.savefig('../plots/full.cluster.inspect.png')

/users/daniel.malzl/.local/lib/python3.9/site-packages/scanpy/plotting/_utils.py:432: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[value_to_plot + '_colors'] = colors_list
/users/daniel.malzl/.local/lib/python3.9/site-packages/scanpy/plotting/_tools/scatterplots.py:394: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(
/users/daniel.malzl/.local/lib/python3.9/site-packages/scanpy/plotting/_utils.py:432: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[value_to_plot + '_colors'] = colors_list
/users/daniel.malzl/.local/lib/python3.9/site-packages/scanpy/plotting/_tools/scatterplots.py:394: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(
/users/daniel.malzl/.local/lib/python3.9/site-packages/scanpy/plotting/_utils.py:432: ImplicitModification

In [3]:
resolutions = {
    'tissue.scps': 0.1,
    'tissue.uc': 0.4,
    'tissue.ad': 0.25,
    'pbmc.scps': 0.3
}
for k, resolution in resolutions.items():
    d = integration_results[k]
    sc.tl.leiden(
        d['data'], 
        key_added = f'leiden_scvi_{resolution}',
        resolution = resolution
    )

In [4]:
cluster_keys = {k: f'leiden_scvi_{r}' for k, r in resolutions.items()}
fig, axs = plot.integrate.plot_clustering_and_expression(
    integration_results,
    cluster_keys,
    ['CD3D', 'FOXP3'],
    [
        dict(size = 10, vmax = None, legend_loc = 'on data'),
        dict(size = 10, vmax = 10),
        dict(size = 10, vmax = 1)
    ],
    data_key = 'data'
)
fig.savefig('../plots/full.cluster.results.png')

/users/daniel.malzl/.local/lib/python3.9/site-packages/scanpy/plotting/_utils.py:432: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[value_to_plot + '_colors'] = colors_list
/users/daniel.malzl/.local/lib/python3.9/site-packages/scanpy/plotting/_tools/scatterplots.py:394: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(
/users/daniel.malzl/.local/lib/python3.9/site-packages/scanpy/plotting/_utils.py:432: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[value_to_plot + '_colors'] = colors_list
/users/daniel.malzl/.local/lib/python3.9/site-packages/scanpy/plotting/_tools/scatterplots.py:394: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(
/users/daniel.malzl/.local/lib/python3.9/site-packages/scanpy/plotting/_utils.py:432: ImplicitModification

In [5]:
tcell_clusters = {
    'tissue.scps': '1',
    'tissue.uc': '0,2,6',
    'tissue.ad': '1,10',
    'pbmc.scps': '0,3,6'
}

tcell_clusters = {
    k: [c for c in cs.split(',')] for k, cs in tcell_clusters.items()
}

In [6]:
for k, d in integration_results.items():
    clustering = cluster_keys[k]
    d['data'].obs['coarse_cell_types'] = d['data'].obs[clustering].apply(
        lambda x: 'Tcell' if x in tcell_clusters[k] else 'other'
    )
    print(k, d['data'].obs.groupby('coarse_cell_types').count().iloc[:, 0])
    
fig, axs = plot.integrate.plot_clustering_and_expression(
    integration_results,
    {k: 'coarse_cell_types' for k in integration_results.keys()},
    ['CD3D', 'FOXP3'],
    [
        dict(size = 10, vmax = None),
        dict(size = 10, vmax = 10),
        dict(size = 10, vmax = 1)
    ],
    data_key = 'data'
)
fig.savefig('../plots/full.cluster.tcell.png')

tissue.scps coarse_cell_types
Tcell     62462
other    270084
Name: sample_id, dtype: int64
tissue.ad coarse_cell_types
Tcell     61787
other    257189
Name: sample_id, dtype: int64
tissue.uc coarse_cell_types
Tcell    46867
other    32511
Name: sample_id, dtype: int64
pbmc.scps coarse_cell_types
Tcell    18557
other    21591
Name: sample_id, dtype: int64


/users/daniel.malzl/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1301: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/users/daniel.malzl/.local/lib/python3.9/site-packages/scanpy/plotting/_tools/scatterplots.py:394: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(
/users/daniel.malzl/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1301: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/users/daniel.malzl/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1301: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/users/daniel.malzl/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1301: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df

In [ ]:
for k, d in integration_results.items():
    d['data'].write(
        f'../data/{k}.integrated.clustered.h5ad'
    )